# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

E. Schinnerer  ->  E. Schinnerer  |  ['E. Schinnerer']
M. Flock  ->  M. Flock  |  ['M. Flock']
A. Gould  ->  A. Gould  |  ['A. Gould']
S. Li  ->  S. Li  |  ['S. Li']
C. Jaeger  ->  C. Jaeger  |  ['C. Jaeger']
S. Ghosh  ->  S. Ghosh  |  ['S. Ghosh']
Arxiv has 79 new papers today
          6 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/6 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2310.18407


extracting tarball to tmp_2310.18407...

 done.


/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3548: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3548: LatexWarning: Found documentclass in tmp_2310.18407/main_v1.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:399: LatexWarning: Latex injecting: 'Sections/appendix' from 'tmp_2310.18407/Sections/appendix.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:399: LatexWarning: Latex injecting: 'Sections/05_conclusion' from 'tmp_2310.18407/Sections/05_conclusion.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.18

...

 done.


C. Jäger  ->  C. Jäger  |  ['C. Jäger']


Found 39 bibliographic references in tmp_2310.19497/ms.bbl.
Retrieving document from  https://arxiv.org/e-print/2310.19627


extracting tarball to tmp_2310.19627...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2310.18407-b31b1b.svg)](https://arxiv.org/abs/arXiv:2310.18407) | **Resolved low-J $^{12}$CO excitation at 190 parsec resolution across NGC  2903 and NGC 3627**  |
|| J. S. d. Brok, et al. -- incl., <mark>E. Schinnerer</mark> |
|*Appeared on*| *2023-10-31*|
|*Comments*| *accepted for publication in MNRAS, 17 pages, 16 figures*|
|**Abstract**| The low-$J$ rotational transitions of $^{12}$CO are commonly used to trace the distribution of molecular gas in galaxies. Their ratios are sensitive to excitation and physical conditions in the molecular gas. Spatially resolved studies of CO ratios are still sparse and affected by flux calibration uncertainties, especially since most do not have high angular resolution or do not have short-spacing information and hence miss any diffuse emission. We compare the low-$J$ CO ratios across the disk of two massive, star-forming spiral galaxies NGC2903 and NGC3627 to investigate whether and how local environments drive excitation variations at GMC scales. We use Atacama Large Millimeter Array (ALMA) observations of the three lowest-$J$ CO transitions at a common angular resolution of 4$''$ (190pc). We measure median line ratios of $R_{21}=0.67^{+0.13}_{-0.11}$, $R_{32}=0.33^{+0.09}_{-0.08}$, and $R_{31}=0.24^{+0.10}_{-0.09}$ across the full disk of NGC3627. We see clear CO line ratio variation across the galaxy consistent with changes in temperature and density of the molecular gas. In particular, toward the center, $R_{21}$, $R_{32}$, and $R_{31}$ increase by 35\%, 50\%, and 66\%, respectively compared to their average disk values. The overall line ratio trends suggest that CO(3-2) is more sensitive to changes in the excitation conditions than the two lower-$J$ transitions. Furthermore, we find a similar radial $R_{32}$ trend in NGC2903, albite a larger disk-wide average of $\langle R_{32}\rangle=0.47^{+0.14}_{-0.08}$. We conclude that the CO low-$J$ line ratios vary across environments in such a way that they can trace changes in the molecular gas conditions, with the main driver being changes in temperature. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2310.18484-b31b1b.svg)](https://arxiv.org/abs/arXiv:2310.18484) | **Kinematic signatures of planet-disk interactions in VSI-turbulent  protoplanetary disks**  |
|| M. Barraza-Alfaro, <mark>M. Flock</mark>, T. Henning |
|*Appeared on*| *2023-10-31*|
|*Comments*| *Accepted for publication in Astronomy & Astrophysics. 27 pages, 17 figures and 2 tables*|
|**Abstract**| Context. Planets are thought to form inside weakly ionized regions of protoplanetary disks, where turbulence creates ideal conditions for solid growth. However, the nature of this turbulence is still uncertain. In this zone, vertical shear instability (VSI) can operate, inducing a low level of gas turbulence and large-scale motions. Resolving kinematic signatures of VSI may reveal the origin of turbulence in planet-forming disks. However, an exploration of kinematic signatures of the interplay between VSI and forming planets is needed for a correct interpretation of radio interferometric observations. Robust detection of VSI would lead to a deeper understanding of the impact of gas turbulence on planet formation. Aims. The goal of this study is to explore the effect of VSI on the disk substructures triggered by an embedded massive planet. We focus on the impact of this interplay on CO kinematic observations with ALMA. Methods. We conduct global 3D hydrodynamical simulations of VSI-unstable disks with and without embedded massive planets, exploring Saturn- and Jupiter-mass cases. We study the effect of planets on the VSI gas dynamics, comparing with viscous disks. Post-processing the simulations with a radiative transfer code, we examine the kinematic signatures expected in CO molecular line emission, varying disk inclination. Further, we simulate ALMA high-resolution observations to test the observability of VSI and planetary signatures. Results. The embedded planet dampens the VSI along a radial region, most effective at the disk midplane. For the Saturn case, the VSI modes are distorted by the planet's spirals producing mixed kinematic signatures. For the Jupiter case, the planet's influence dominates the disk gas kinematics. Conclusions. The presence of massive embedded planets can weaken the VSI large-scale gas flows, limiting its observability in CO kinematic observations. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2310.19497-b31b1b.svg)](https://arxiv.org/abs/arXiv:2310.19497) | **Interstellar carbonaceous dust erosion induced by X-ray irradiation of  water ice in star-forming regions**  |
|| K.-J. Chuang, et al. -- incl., <mark>C. Jaeger</mark> |
|*Appeared on*| *2023-10-31*|
|*Comments*| *12 pages, 9 figures, 1 table*|
|**Abstract**| The chemical inventory of protoplanetary midplanes is the basis for forming planetesimals. Among them, solid-state reactions based on CO/CO$_2$ toward molecular complexity on interstellar dust grains have been studied in theoretical and laboratory work. In this work, the erosion of C dust grains induced by X-ray irradiation of H$_2$O ice was systematically investigated for the first time. The work aims to provide a better understanding of the reaction mechanism using selectively isotope-labeled oxygen/carbon species in kinetic analysis. Ultrahigh vacuum experiments were performed to study the interstellar ice analog on sub-$\mu$m thick C dust at $\sim$13~K. H$_2$O or O$_2$ ice was deposited on the pre-synthesized amorphous C dust and exposed to soft X-ray photons (250--1250~eV). Fourier-transform infrared spectroscopy was used to monitor in situ the newly formed species as a function of the incident photon fluence. Field emission scanning electron microscopy was used to monitor the morphological changes of (non-)eroded carbon samples. The X-ray processing of the ice/dust interface leads to the formation of CO$_2$, which further dissociates and forms CO. Carbonyl groups are formed by oxygen addition to grain surfaces and are confirmed as intermediate species in the formation process. The yields of CO and CO$_2$ were found to be dependent on the thickness of the carbon layer. The astronomical relevance of the experimental findings is discussed. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2310.19164-b31b1b.svg)](https://arxiv.org/abs/arXiv:2310.19164) | **Industrial-Scale Mass Measurements of Isolated Black Holes**  |
|| <mark>A. Gould</mark> |
|*Appeared on*| *2023-10-31*|
|*Comments*| *39 pages, 7 figures, to be submitted to AAS Journals*|
|**Abstract**| I show that industrial-scale mass measurement of isolated black holes (BHs) can be achieved by combining a high-cadence, wide-field microlensing survey such as KMTNet, observations from a parallax satellite in solar orbit, and VLTI GRAVITY+ interferometry. I show that these can yield precision measurements of microlens parallaxes down to $\pi_{\rm E}\sim 0.01$ and Einstein radii down to $\theta_{\rm E}\sim 1\,$mas. These limits correspond to BH masses $M\sim 12\,M_\odot$, deep in the Galactic bulge, with lens-source separations of $D_{LS}\sim 0.6\,$kpc, and they include all BHs in the Galactic disk. I carry out detailed analyses of simulations that explore many aspects of the measurement process, including the decisions on whether to carry out VLTI measurements for each long-event candidate. I show that the combination of ground-based and space-based light curves of BH events will automatically exclude the spurious "large parallax" solutions that arise from the standard (Refsdal 1966) analysis, except for the high-magnification events, for which other methods can be applied. The remaining two-fold degeneracy can always be broken by conducting a second VLTI measurement, and I show how to identify the relatively rare cases that this is required. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2310.19228-b31b1b.svg)](https://arxiv.org/abs/arXiv:2310.19228) | **Simultaneous Horizontal and Vertical Oscillation of a Quiescent Filament  observed by CHASE and SDO**  |
|| J. Dai, et al. -- incl., <mark>S. Li</mark> |
|*Appeared on*| *2023-10-31*|
|*Comments*| **|
|**Abstract**| In this paper, we present the imaging and spectroscopic observations of the simultaneous horizontal and vertical large-amplitude oscillation of a quiescent filament triggered by an EUV wave on 2022 October 02. Particularly, the filament oscillation involved winking phenomenon in Ha images and horizontal motions in EUV images. Originally, a filament and its overlying loops across AR 13110 and 13113 erupted with a highly inclined direction, resulting in an X1.0 flare and a non-radial CME. The fast lateral expansion of loops excited an EUV wave and the corresponding Moreton wave propagating northward. Once the EUV wavefront arrived at the quiescent filament, the filament began to oscillate coherently along the horizontal direction and the winking filament appeared concurrently in Ha images. The horizontal oscillation involved an initial amplitude of 10.2 Mm and a velocity amplitude of 46.5 km/s, lasting for 3 cycles with a period of 18.2 minutes and a damping time of 31.1 minutes. The maximum Doppler velocities of the oscillating filament are 18 km/s (redshift) and 24 km/s (blueshift), which was derived from the spectroscopic data provided by CHASE/HIS. The three-dimensional velocity of the oscillation is determined to be 50 km/s at an angle of 50 to the local photosphere plane. Based on the wave-filament interaction, the minimum energy of the EUV wave is estimated to be 2.7 10 20 J. Furthermore, this event provides evidence that Moreton wavesshould be excited by the highly inclined eruptions. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2310.19627-b31b1b.svg)](https://arxiv.org/abs/arXiv:2310.19627) | **Foreground removal with ILC methods for AliCPT-1**  |
|| J. Dou, <mark>S. Ghosh</mark>, L. Santos, W. Zhao |
|*Appeared on*| *2023-10-31*|
|*Comments*| **|
|**Abstract**| One of the main goals of most future CMB experiments is the precise measurement of CMB B-mode polarization, whose major obstacle is the Galactic foregrounds. In this paper, we evaluate the foreground cleaning performance of the variants of the ILC method on partial sky B-modes and analyze the main sources of biases on the BB power spectrum. Specially, we compare the NILC, the cILC (in three domains) and the cMILC methods for AliCPT-1 simulations. We find that the cILC methods implemented in harmonic space and needlet space are both competent to clean different models of foregrounds, which bias the tensor-to-scalar ratio about 0.008 at maximum, and constrain the tensor-to-scalar ratio to r<0.043 (95\%CL) for the AliCPT-1 configuration. We also note that the deviation of the estimated noise bias from the actual one for ILC, dubbed the noise bias error (NBE) in this paper, might make significant effects on the power spectrum for a small footprint and low signal-to-noise ratio CMB experiment. We finally obtain its relation with respect to the noise residual which fits well with the simulated results. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2310.18407.md
    + _build/html/tmp_2310.18407/./Figures/ratio_vs_ratio.png
    + _build/html/tmp_2310.18407/./Figures/full_correlations.png
    + _build/html/tmp_2310.18407/./Figures/maps_ratio.png
exported in  _build/html/2310.18484.md
    + _build/html/tmp_2310.18484/./Plots_Main/Simulations_VSI_Midplane.png
    + _build/html/tmp_2310.18484/./Plots_Main/Simulations_VSI_Surface.png
    + _build/html/tmp_2310.18484/./Plots_Main/Residuals_TwoLayers_VSI.png
exported in  _build/html/2310.19497.md
    + _build/html/tmp_2310.19497/./Figures/Figure02.png
    + _build/html/tmp_2310.19497/./Figures/Figure01.png
    + _build/html/tmp_2310.19497/./Figures/FigureA2.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{  \Hi      }{\ifmmode{\rm H} \textsc{i} \else H \textsc{i}\fi}$
$\newcommand{\delim}{{-}}$
$\newcommand{\figureautorefname}{Fig.}$
$\newcommand{\equationautorefname}{Eq.}$
$\newcommand{\sectionautorefname}{Section}$
$\newcommand{\subsectionautorefname}{Section}$
$\newcommand{\subsubsectionautorefname}{Section}$
$\newcommand{\appendixautorefname}{Appendix}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# Resolved low-$J$ $^{12}$CO excitation at $190$ parsec resolution across NGC 2903 and NGC 3627

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2310.18407-b31b1b.svg)](https://arxiv.org/abs/2310.18407)<mark>Appeared on: 2023-10-31</mark> -  _accepted for publication in MNRAS, 17 pages, 16 figures_

</div>
<div id="authors">

J. S. d. Brok, et al. -- incl., <mark>E. Schinnerer</mark>

</div>
<div id="abstract">

**Abstract:** The low- $J$ rotational transitions of $\chem{^{12}CO}$ are commonly used to trace the distribution of molecular gas in galaxies. Their ratios are sensitive to excitation and physical conditions in the molecular gas. Spatially resolved studies of CO ratios are still sparse and affected by flux calibration uncertainties, especially since most do not have high angular resolution or do not have short-spacing information and hence miss any diffuse emission. We compare the low- $J$ CO ratios across the disk of two massive, star-forming spiral galaxies NGC 2903 and NGC 3627 to investigate whether and how local environments drive excitation variations at GMC scales. We use Atacama Large Millimeter Array (ALMA) observations of the three lowest- $J$ CO transitions at a common angular resolution of 4 $"$ (190 pc).We measure median line ratios of $R_{21}=0.67^{+0.13}_{-0.11}$ , $R_{32}=0.33^{+0.09}_{-0.08}$ , and $R_{31}=0.24^{+0.10}_{-0.09}$ across the full disk of NGC 3627. We see clear CO line ratio variation across the galaxy consistent with changes in temperature and density of the molecular gas. In particular, toward the center, $R_{21}$ , $R_{32}$ , and $R_{31}$ increase by 35 \% , 50 \% , and 66 \% , respectively compared to their average disk values.The overall line ratio trends suggest that CO(3-2) is more sensitive to changes in the excitation conditions than the two lower- $J$ transitions. Furthermore, we find a similar radial $R_{32}$ trend in NGC 2903, albite a larger disk-wide average of $\langle R_{32}\rangle=0.47^{+0.14}_{-0.08}$ . We conclude that the CO low- $J$ line ratios vary across environments in such a way that they can trace changes in the molecular gas conditions, with the main driver being changes in temperature.

</div>

<div id="div_fig1">

<img src="tmp_2310.18407/./Figures/ratio_vs_ratio.png" alt="Fig4" width="100%"/>

**Figure 4. -** ** CO ratio-to-ratio comparison in NGC 3627**. The panels depict the three permutations of comparing $R_{21}$, $R_{32}$, and $R_{31}$ to each other. The light-colored points have S/N${>}5$ in all lines. The filled circles are the binned trend. The filled squares represent the result when computing the line ratio from the spectral stacks by SFR (using all lines of sight irrespective of S/N). For reference, the red line indicates the 1:1 relation. We perform a linear regression to the stacked data points, represented as a black dashed line. We note that the axes of the different panels are correlated. The sense of the implicit correlation between the plotted variables is indicated in each panel.  (*fig:ratio_vs_ratio*)

</div>
<div id="div_fig2">

<img src="tmp_2310.18407/./Figures/full_correlations.png" alt="Fig13" width="100%"/>

**Figure 13. -** ** CO line ratio trends with $\Sigma_{\rm SFR**$ and dust tracers $I_{7.7 \mu\rm m}$ (PAH) and  $I_{7.7 \mu\rm m}$ (hot dust)}. The different panels show the trends of $R_{21}$(_top_), $R_{32}$(_middle_), and $R_{31}$(_bottom_) for the different environments. The orange hexagonal bins represent the 2D histogram of $\rm S/N{\ge}5$ sightlines. We note that the stacked trends also include the sightlines $\rm S/N{<}5$.
     (*fig:ratio_comp*)

</div>
<div id="div_fig3">

<img src="tmp_2310.18407/./Figures/maps_ratio.png" alt="Fig11" width="100%"/>

**Figure 11. -** ** CO line ratio trends with galactocentric radius in NGC 3627**. The panels on the left show the radial trend of $R_{21}$(_top_), $R_{32}$(_center_), and $R_{31}$(_bottom_). The red-shaded region shows the radial extent beyond which our sampling is limited by the map size. The large filled circles show the stacked (we stack datapoints irrespective of the different regions).  The sightlines are color-coded by different environmental regions. For reference, the gray ellipses indicate deprojected radial distances of 1, 2, 3, 4, 5, and 6 kpc. The extent of the different regions is shown in the CO line ratio maps on the right-hand side. For these panels, we only select data points where line intensities have ${\rm S/N}{>5}$ for both the denominator and numerator of the ratio. (*fig:ngc3627_ratio*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2310.18407"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\sgn}{\text{sgn}}$</div>



<div id="title">

# Kinematic signatures of planet-disk interactions$\in VSI-turbulent protoplanetary disks$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2310.18484-b31b1b.svg)](https://arxiv.org/abs/2310.18484)<mark>Appeared on: 2023-10-31</mark> -  _Accepted for publication in Astronomy & Astrophysics. 27 pages, 17 figures and 2 tables_

</div>
<div id="authors">

M. Barraza-Alfaro, <mark>M. Flock</mark>, T. Henning

</div>
<div id="abstract">

**Abstract:** Planets are thought to form inside weakly ionized regions of protoplanetary disks, in which turbulence creates ideal conditions for solid growth. However, the nature of this turbulence is still uncertain. In fast cooling parts of this zone the vertical shear instability (VSI) can operate, inducing a low level of gas turbulence and large-scale gas motions. Resolving kinematic signatures of active VSI could reveal the origin of turbulence in planet-forming disk regions. However, an exploration of kinematic signatures of the interplay between VSI and forming planets is needed for a correct interpretation of radio interferometric observations. A robust detection of VSI would open the door for a deeper understanding of the impact of gas turbulence on planet formation. The objective of this study is to explore the effect of the VSI on the disk substructures triggered by an embedded fairly massive planet. We will focus on the impact of this interplay on CO kinematic observations with the ALMA interferometer. We conducted global 3D hydrodynamical simulations of VSI-unstable disks with and without embedded massive planets, exploring Saturn- and Jupiter-mass cases. We studied the effect of planets on the VSI gas dynamics, comparing with viscous disks.   Post-processing the simulations with a radiative transfer code, we examined the kinematic signatures expected in CO molecular line emission, varying disk inclination. Further, we simulate deep ALMA high-resolution observations of our synthetic images, to test the observability of VSI and planetary signatures. The embedded planet produces a damping of the VSI along a radial region, most effective at the disk midplane. For the Saturn case, the VSI modes are distorted by the planet's spirals producing mixed kinematic signatures. For the Jupiter case, the planet's influence dominates the overall disk gas kinematics. The presence of massive planets embedded in the disk can weaken the VSI large-scale gas flows, limiting its observability in CO kinematic observations with ALMA.

</div>

<div id="div_fig1">

<img src="tmp_2310.18484/./Plots_Main/Simulations_VSI_Midplane.png" alt="Fig1" width="100%"/>

**Figure 1. -** Snapshots of the VSI-unstable hydrodynamical simulations at the disk midplane. From top to bottom: simulation without an embedded planet, with an embedded Saturn-mass planet, and with an Jupiter-mass planet. From left to right: gas density relative to the initial value ($\rho/\rho_0$), radial velocity field ($v_r$), meridional velocity field ($v_{\theta}$), and azimuthal velocity deviations from Keplerian rotation ($v_{\phi}-v_{\rm Kep}$).
The included planets are orbiting at $100$ au from the central star, marked with dotted lines in the second and third rows. The dashed lines in the first panel indicate the buffer zones applied in the hydro simulations. (*fig:simulationsvsimidplane*)

</div>
<div id="div_fig2">

<img src="tmp_2310.18484/./Plots_Main/Simulations_VSI_Surface.png" alt="Fig2" width="100%"/>

**Figure 2. -** Snapshots of the VSI-unstable hydrodynamical simulations at the disk surface. Same as Figure \ref{fig:simulationsvsimidplane}, but obtained at 3 pressure scale heights from the disk midplane ($Z\sim 3H$). (*fig:simulationsvsisurface*)

</div>
<div id="div_fig3">

<img src="tmp_2310.18484/./Plots_Main/Residuals_TwoLayers_VSI.png" alt="Fig9" width="100%"/>

**Figure 9. -** Deviations from Keplerian rotation for $^{13}$CO$(3-2)$ synthetic images of the post-processed simulations, for a VSI unstable disk without embedded planets. In the first row, from left to right, we show models including the full disk, only the disk hemisphere closer to the observer (front side), and the farther hemisphere of the disk (back side), for a disk inclination of 30 degrees. In the second row, a deprojected view of the residual maps (first row) is shown. The fourth panel, shows a normalized square root of the product of the front and back sides deprojected residual maps, times the sign of the initial multiplication of both residuals. A ring of interest is enclosed by black-dashed lines, drawn at 1.55 and 1.85 arcsec. The black solid line along the semi-minor axis in the non-Keplerian residuals shows the line of zero gas velocity projected into the line-of-sight. (*fig:residualstwolayersvsi*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2310.18484"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$</div>



<div id="title">

# Interstellar carbonaceous dust erosion induced by X-ray irradiation of water ice in star-forming regions

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2310.19497-b31b1b.svg)](https://arxiv.org/abs/2310.19497)<mark>Appeared on: 2023-10-31</mark> -  _12 pages, 9 figures, 1 table_

</div>
<div id="authors">

K.-J. Chuang, et al. -- incl., <mark>C. Jäger</mark>

</div>
<div id="abstract">

**Abstract:** The chemical inventory of protoplanetary midplanes is the basis for forming planetesimals. Among them, solid-state reactions based on CO/$CO_2$ toward molecular complexity on interstellar dust grains have been studied in theoretical and laboratory work.The physicochemical interactions between ice, constituted mainly of $H_2$ O, and dust surfaces are limited to a few experimental studies focusing on vacuum ultraviolet and cosmic-ray processing.In this work, the erosion of C dust grains induced by X-ray irradiation of $H_2$ O ice was systematically investigated for the first time. The work aims to provide a better understanding of the reaction mechanism using selectively isotope-labeled oxygen/carbon species in kinetic analysis.Ultrahigh vacuum experiments were performed to study the interstellar ice analog on sub- $\mu$ m thick C dust at $\sim$ 13 K. $H_2$ O or $O_2$ ice was deposited on the pre-synthesized amorphous C dust and exposed to soft X-ray photons (250--1250 eV). Fourier-transform infrared spectroscopy was used to monitor in situ the newly formed species as a function of the incident photon fluence. Field emission scanning electron microscopy was used to monitor the morphological changes of (non-)eroded carbon samples.The X-ray processing of the ice/dust interface leads to the formation of $CO_2$ , which further dissociates and forms CO.Carbonyl groups are formed by oxygen addition to grain surfaces and are confirmed as intermediate species in the formation process. The yields of CO and $CO_2$ were found to be dependent on the thickness of the carbon layer.The astronomical relevance of the experimental findings is discussed.

</div>

<div id="div_fig1">

<img src="tmp_2310.19497/./Figures/Figure02.png" alt="Fig8" width="100%"/>

**Figure 8. -** Left: the IR difference spectra obtained before and after X-ray irradiation of (a) $^{(16)}$O$_{2}$ and (b) $^{18}$O$_{2}$ ices on the same a-$^{13}$C dust analog as well as (c) $H_2$O on a-$^{13}$C dust analog at 15 K for 60 minutes. Right: the corresponding residual spectra measured at 300 K. The thickness of a-$^{13}$C dust is $\sim$100 nm, and the deposited abundances of $H_2$O and $^{(16/18)}$O$_{2}$ are $\sim$5 $\times$ 10$^{17}$ molecule cm$^{-2}$. The total fluence of X-ray photons is (7.6--7.9) $\times$ 10$^{18}$ photon cm$^{-2}$. The dashed lines highlight the peak position of $^{13}$C-labeled products, and the shadowed area highlights the regions of the carbonyl group feature. The spectra are offset for clarity.
             (*Fig2*)

</div>
<div id="div_fig2">

<img src="tmp_2310.19497/./Figures/Figure01.png" alt="Fig7" width="100%"/>

**Figure 7. -** Left: the IR difference spectra obtained before and after X-ray irradiation of $H_2$O ice on a-$^{13}$C dust analog at 15 K for 60 minutes. The thickness of a-$^{13}$C dust is $\sim$100 nm, and the deposited abundance of $H_2$O is $\sim$5 $\times$ 10$^{17}$ molecule cm$^{-2}$. The dashed lines highlight the peak position of $^{13}$C-labeled products. The spectra are offset for clarity. Right: abundance evolution of the product $^{13}$$CO_2$(solid) and $^{13}$CO (open) obtained from the X-ray irradiation of $H_2$O ice on the same a-$^{13}$C dust over X-ray photons fluence of (7.6--8.0) $\times$ 10$^{18}$ photon cm$^{-2}$(i.e., (7.0--7.4) $\times$ 10$^{20}$ eV cm$^{-2}$). The dashed lines show the linear fit for the initial formation rate. (*Fig1*)

</div>
<div id="div_fig3">

<img src="tmp_2310.19497/./Figures/FigureA2.png" alt="Fig4" width="100%"/>

**Figure 4. -** The X-ray photon flux and $H_2$O transmittance as a function of photon energy. (*FigA1*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2310.19497"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

312  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

7  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

5  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
